In [ ]:
import pandas as pd

traffic_data_path = '/content/drive/MyDrive/Colab Notebooks/traffic_hourly_avg_filtered_districts.csv'

df = pd.read_csv(traffic_data_path)

df.columns = [c.lower().strip() for c in df.columns]

average_speed = df['average_speed'].mean()
print(f"Average speed for all districts & hours: {average_speed:.2f} km/h")

avg_speed_by_district = df.groupby('district')['average_speed'].mean()
print("\nAverage speed by district:")
print(avg_speed_by_district)


Tüm bölgeler & saatler için ortalama hız: 44.96 km/h

Her district için ortalama hız:
district
Beşiktaş     36.550254
Kağıthane    39.004765
Sarıyer      49.835363
Şişli        31.271707
Name: average_speed, dtype: float64


In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

fire_station_file_path = '/content/drive/MyDrive/Colab Notebooks/filtered_itfaiye.csv'
demand_file_path = '/content/drive/MyDrive/Colab Notebooks/final_critical_demand_zones_80_for_success.csv'

MEAN_SPEED_KPH = 44.96

df_stations = pd.read_csv(fire_station_file_path)
df_stations['latitude'] = df_stations['Koordinat (X)']
df_stations['longitude'] = df_stations['Koordinat (Y)']
station_names = df_stations['Birim'].tolist()

df_demands = pd.read_csv(demand_file_path)
df_demands['latitude'] = df_demands['LATITUDE']
df_demands['longitude'] = df_demands['LONGITUDE']
demand_ids = df_demands['GEOHASH'].tolist()

num_stations = len(df_stations)
num_demands = len(df_demands)

Tij_matrix = np.zeros((num_stations, num_demands))

for i in range(num_stations):
    station_coord = (df_stations.loc[i, 'latitude'], df_stations.loc[i, 'longitude'])
    for j in range(num_demands):
        demand_coord = (df_demands.loc[j, 'latitude'], df_demands.loc[j, 'longitude'])
        distance_m = geodesic(station_coord, demand_coord).meters
        time_sec = distance_m / (MEAN_SPEED_KPH * 1000 / 3600)
        Tij_matrix[i, j] = time_sec

Tij_df = pd.DataFrame(Tij_matrix, index=station_names, columns=demand_ids)
Tij_df.to_csv('/content/drive/MyDrive/Colab Notebooks/matrix/STATIC_Tij_matrix_80_for_success.csv', encoding='utf-8-sig')
print("Static Tij matrix created and saved successfully!")


Statik Tij matrisi başarıyla oluşturuldu ve kaydedildi!


In [ ]:
import pandas as pd
import numpy as np

fire_station_file_path = '/content/drive/MyDrive/Colab Notebooks/filtered_itfaiye.csv'
selected_demand_zones_file_path = '/content/drive/MyDrive/Colab Notebooks/selected_80_demand_zones.csv'
STATIC_AVG_SPEED_KPH = 44.96

def haversine(lon1, lat1, lon2, lat2):
    R = 6371
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

fire_station_df = pd.read_csv(fire_station_file_path)
demand_df = pd.read_csv(selected_demand_zones_file_path)

if 'Koordinat (X)' in fire_station_df.columns and 'Koordinat (Y)' in fire_station_df.columns:
    fire_station_lat = fire_station_df['Koordinat (X)'].astype(float).values
    fire_station_lon = fire_station_df['Koordinat (Y)'].astype(float).values
else:
    fire_station_lat = fire_station_df['latitude'].astype(float).values
    fire_station_lon = fire_station_df['longitude'].astype(float).values

if 'LATITUDE' in demand_df.columns and 'LONGITUDE' in demand_df.columns:
    demand_lat = demand_df['LATITUDE'].astype(float).values
    demand_lon = demand_df['LONGITUDE'].astype(float).values
else:
    demand_lat = demand_df['latitude'].astype(float).values
    demand_lon = demand_df['longitude'].astype(float).values

num_stations = len(fire_station_lat)
num_demands = len(demand_lat)
Tij_matrix = np.zeros((num_stations, num_demands))

for i in range(num_stations):
    for j in range(num_demands):
        dist_km = haversine(fire_station_lon[i], fire_station_lat[i], demand_lon[j], demand_lat[j])
        travel_time_hr = dist_km / STATIC_AVG_SPEED_KPH
        Tij_matrix[i, j] = travel_time_hr * 3600

station_names = fire_station_df['Birim'].tolist() if 'Birim' in fire_station_df.columns else [f'Station_{i}' for i in range(num_stations)]
demand_ids = demand_df['GEOHASH'].tolist() if 'GEOHASH' in demand_df.columns else [f'Demand_{j}' for j in range(num_demands)]

Tij_df = pd.DataFrame(Tij_matrix, index=station_names, columns=demand_ids)
Tij_df.to_csv('/content/drive/MyDrive/Colab Notebooks/matrix/STATIC_kmeans80_Tij_matrix.csv', encoding='utf-8-sig')

print("Static (Euclidean + constant speed) Tij matrix created and saved.")


Statik (Öklid+Sabit hız) Tij matrisi oluşturuldu ve kaydedildi.
